In [3]:
# Cell 1: Setup and Imports
import json
import pandas as pd
from datetime import datetime, timedelta
import sys
sys.path.append('/opt/airflow')
from utils.report_utils import query_order_api, generate_pdf_report

# For inline display (if you have PDF viewing capability)
from IPython.display import IFrame, display
import os

print("Environment ready!")

ModuleNotFoundError: No module named 'utils'

In [ ]:
# Cell 2: Create Sample Order Data
# Simulate MAO Order Search API response
sample_orders = [
    {
        "OrderId": "ORD-2024-001",
        "OrderDate": "2024-11-20T08:30:00Z",
        "CustomerName": "Acme Pet Supplies",
        "Status": "Released",
        "TotalItems": 15,
        "TotalValue": 1250.50,
        "ShipNode": "DC01",
        "Priority": "Standard"
    },
    {
        "OrderId": "ORD-2024-002",
        "OrderDate": "2024-11-20T09:15:00Z",
        "CustomerName": "Paws & Claws Inc",
        "Status": "BackOrdered",
        "TotalItems": 8,
        "TotalValue": 675.25,
        "ShipNode": "DC01",
        "Priority": "Expedited"
    },
    {
        "OrderId": "ORD-2024-003",
        "OrderDate": "2024-11-20T10:00:00Z",
        "CustomerName": "Pet Paradise",
        "Status": "Released",
        "TotalItems": 22,
        "TotalValue": 2100.00,
        "ShipNode": "DC02",
        "Priority": "Standard"
    },
    {
        "OrderId": "ORD-2024-004",
        "OrderDate": "2024-11-20T11:30:00Z",
        "CustomerName": "Furry Friends Co",
        "Status": "Shipped",
        "TotalItems": 12,
        "TotalValue": 890.75,
        "ShipNode": "DC01",
        "Priority": "Standard"
    }
]

# Convert to DataFrame for easier manipulation
df_orders = pd.DataFrame(sample_orders)
print(f"Sample dataset: {len(df_orders)} orders")
df_orders.head()

In [ ]:
# Cell 3: Load Your Report Configuration
report_config = {
    "report_id": "daily_order_summary",
    "name": "Daily Order Summary",
    "description": "Summary of all orders processed in the last 24 hours",
    "report_fields": [
        "OrderId", "OrderDate", "CustomerName", "Status", "TotalItems", "TotalValue"
    ],
    "summary_fields": [
        {"field": "TotalValue", "operation": "sum", "label": "Total Revenue"},
        {"field": "TotalItems", "operation": "sum", "label": "Total Items"},
        {"field": "OrderId", "operation": "count", "label": "Order Count"}
    ],
    "formatting": {
        "currency_fields": ["TotalValue"],
        "date_fields": ["OrderDate"],
        "title_font_size": 16,
        "header_color": "#2C5F8D"
    }
}

print(json.dumps(report_config, indent=2))

In [ ]:
# Cell 4: Calculate Summary Statistics
def calculate_summaries(df, summary_fields):
    """Calculate summary statistics based on configuration"""
    summaries = {}
    
    for summary in summary_fields:
        field = summary['field']
        operation = summary['operation']
        label = summary['label']
        
        if operation == 'sum':
            summaries[label] = df[field].sum()
        elif operation == 'count':
            summaries[label] = df[field].count()
        elif operation == 'avg' or operation == 'mean':
            summaries[label] = df[field].mean()
        elif operation == 'group':
            summaries[label] = df[field].value_counts().to_dict()
    
    return summaries

# Test the summary calculations
summaries = calculate_summaries(df_orders, report_config['summary_fields'])
print("Calculated Summaries:")
for label, value in summaries.items():
    if isinstance(value, float):
        print(f"  {label}: ${value:,.2f}" if 'Revenue' in label else f"  {label}: {value:.2f}")
    else:
        print(f"  {label}: {value}")

In [ ]:
# Cell 5: Create Mock generate_pdf_report Function (if you need to iterate on logic)
def mock_generate_pdf_report(data, config, output_path):
    """
    Mock version to test report generation logic without full PDF dependencies
    Returns the structured data that would go into the PDF
    """
    from reportlab.lib.pagesizes import letter
    from reportlab.lib import colors
    from reportlab.lib.units import inch
    from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
    from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
    
    # Create PDF
    doc = SimpleDocTemplate(output_path, pagesize=letter)
    elements = []
    styles = getSampleStyleSheet()
    
    # Title
    title_style = ParagraphStyle(
        'CustomTitle',
        parent=styles['Heading1'],
        fontSize=config.get('formatting', {}).get('title_font_size', 16),
        textColor=colors.HexColor(config.get('formatting', {}).get('header_color', '#000000')),
        spaceAfter=30
    )
    title = Paragraph(config['name'], title_style)
    elements.append(title)
    
    # Report date
    date_text = f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
    elements.append(Paragraph(date_text, styles['Normal']))
    elements.append(Spacer(1, 0.25*inch))
    
    # Summary Section
    elements.append(Paragraph("<b>Summary Statistics</b>", styles['Heading2']))
    summaries = calculate_summaries(data, config['summary_fields'])
    
    summary_data = [['Metric', 'Value']]
    for label, value in summaries.items():
        if isinstance(value, (int, float)):
            if 'Revenue' in label or 'Value' in label:
                formatted_value = f"${value:,.2f}"
            else:
                formatted_value = f"{value:,.0f}"
        else:
            formatted_value = str(value)
        summary_data.append([label, formatted_value])
    
    summary_table = Table(summary_data, colWidths=[3*inch, 2*inch])
    summary_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 12),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))
    elements.append(summary_table)
    elements.append(Spacer(1, 0.5*inch))
    
    # Detail Section
    elements.append(Paragraph("<b>Order Details</b>", styles['Heading2']))
    
    # Prepare data table
    report_fields = config['report_fields']
    table_data = [report_fields]  # Header row
    
    for _, row in data.iterrows():
        row_data = []
        for field in report_fields:
            value = row[field]
            # Format based on field type
            if field in config.get('formatting', {}).get('currency_fields', []):
                row_data.append(f"${value:,.2f}")
            elif field in config.get('formatting', {}).get('date_fields', []):
                row_data.append(value[:10])  # Just date portion
            else:
                row_data.append(str(value))
        table_data.append(row_data)
    
    # Create table with dynamic column widths
    num_cols = len(report_fields)
    col_width = 6.5*inch / num_cols
    
    detail_table = Table(table_data, colWidths=[col_width]*num_cols)
    detail_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#2C5F8D')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 10),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('FONTSIZE', (0, 1), (-1, -1), 8)
    ]))
    elements.append(detail_table)
    
    # Build PDF
    doc.build(elements)
    
    return {
        'status': 'success',
        'output_path': output_path,
        'records': len(data),
        'summaries': summaries
    }

print("Mock PDF generator ready!")

In [ ]:
# Cell 6: Generate Test Report
output_path = '/tmp/test_daily_order_summary.pdf'

result = mock_generate_pdf_report(
    data=df_orders,
    config=report_config,
    output_path=output_path
)

print(f"Report generated: {result['status']}")
print(f"Output: {result['output_path']}")
print(f"Records: {result['records']}")
print(f"File size: {os.path.getsize(output_path) / 1024:.2f} KB")

In [ ]:
# Cell 7: View the PDF (if running in Jupyter with PDF support)
if os.path.exists(output_path):
    IFrame(output_path, width=800, height=600)
else:
    print(f"PDF not found at {output_path}")

In [ ]:
# Cell 8: Test Different Report Configurations
test_configs = [
    {
        "name": "Exception Orders - Critical Priority",
        "report_fields": ["OrderId", "CustomerName", "ExceptionCode", "ExceptionDesc", "PriorityLevel"],
        "summary_fields": [
            {"field": "ExceptionCode", "operation": "count", "label": "Exception Count"}
        ]
    },
    {
        "name": "Ship Node Performance",
        "report_fields": ["ShipNode", "Status", "TotalItems", "TotalValue"],
        "summary_fields": [
            {"field": "TotalValue", "operation": "sum", "label": "Total Value by Node"},
            {"field": "ShipNode", "operation": "group", "label": "Orders by Ship Node"}
        ]
    }
]

# Generate multiple test reports
for idx, test_config in enumerate(test_configs):
    test_config.update({
        "report_id": f"test_{idx}",
        "formatting": report_config.get("formatting", {})
    })
    
    test_output = f'/tmp/test_report_{idx}.pdf'
    
    try:
        result = mock_generate_pdf_report(df_orders, test_config, test_output)
        print(f"✓ Generated: {test_config['name']} ({os.path.getsize(test_output) / 1024:.1f} KB)")
    except Exception as e:
        print(f"✗ Failed: {test_config['name']} - {str(e)}")

In [ ]:
# Cell 10: Validate Report Fields Against Sample Data
def validate_report_config(data, config):
    """Validate that all configured fields exist in the data"""
    issues = []
    
    # Check report fields
    for field in config['report_fields']:
        if field not in data.columns:
            issues.append(f"Missing field in data: {field}")
    
    # Check summary fields
    for summary in config['summary_fields']:
        field = summary['field']
        if field not in data.columns:
            issues.append(f"Missing summary field in data: {field}")
    
    if issues:
        print("⚠️  Configuration Issues Found:")
        for issue in issues:
            print(f"  - {issue}")
    else:
        print("✓ Configuration validated successfully!")
    
    return len(issues) == 0

# Test validation
validate_report_config(df_orders, report_config)
validate_report_config(df_exceptions, exception_config)

In [ ]:
# Cell 9: Test with Exception Orders (Different Data Pattern)
exception_orders = [
    {
        "OrderId": "ORD-2024-005",
        "OrderDate": "2024-11-20T14:00:00Z",
        "CustomerName": "Problem Pet Store",
        "ExceptionCode": "INV_SHORTAGE",
        "ExceptionDesc": "Inventory shortage for SKU-12345",
        "PriorityLevel": "High",
        "Status": "Hold"
    },
    {
        "OrderId": "ORD-2024-006",
        "OrderDate": "2024-11-20T15:30:00Z",
        "CustomerName": "Late Delivery Co",
        "ExceptionCode": "CARRIER_DELAY",
        "ExceptionDesc": "Carrier reported 2-day delay",
        "PriorityLevel": "Medium",
        "Status": "Exception"
    }
]

df_exceptions = pd.DataFrame(exception_orders)

exception_config = {
    "report_id": "exception_orders",
    "name": "Exception Orders Report",
    "report_fields": ["OrderId", "CustomerName", "ExceptionCode", "ExceptionDesc", "PriorityLevel"],
    "summary_fields": [
        {"field": "PriorityLevel", "operation": "group", "label": "Exceptions by Priority"}
    ],
    "formatting": {
        "date_fields": ["OrderDate"],
        "title_font_size": 16,
        "header_color": "#C41E3A"  # Red for exceptions
    }
}

exception_output = '/tmp/test_exception_report.pdf'
result = mock_generate_pdf_report(df_exceptions, exception_config, exception_output)
print(f"Exception report generated: {result['status']}")